# Consumer expenditure categorization

### Why relational learning matters

This example demonstrates how powerful a real relational learning algorithm can be. Based on a public-domain dataset on consumer behavior, we use a propostionalization algorithm to predict whether purchases were made as a gift. We show that with relational learning, we can get an AUC of over 90%. The generated features would have been impossible to build by hand or by using brute-force approaches.

Summary:

- Prediction type: __Classification model__
- Domain: __Retail__
- Prediction target: __If a purchase is a gift__ 
- Source data: __Relational data set, 4 tables__
- Population size: __398.895__

_Author: Patrick Urbanke_

# Background

Relational learning is one of the most underappreciated fields of machine learning. Even though relational learning is very relevant to many real world data science projects, many data scientists don't even know what relational learning is. 

There are many subdomains of relational learning, but the most important one is extracting features from relational data: Most business data is relational, meaning that it is spread out over several relational tables. However, most machine learning algorithms require that the data be presented in the form of a single flat table. So we need to extract features from our relational data. Some people also call this data wrangling.

Most data scientists we know extract features from relational data manually or by using crude, brute-force approaches (randomly generate thousands of features and then do a feature selection). This is very time-consuming and does not produce good features.

### The challenge

The Consumer Expenditure Data Set is a public domain data set provided by the American Bureau of Labor Statistics (https://www.bls.gov/cex/pumd.htm). It includes the diary entries, where American consumers are asked to keep diaries of the products they have purchased each month.

These consumer goods are categorized using a six-digit classification system the UCC. This system is hierarchical, meaning that every digit represents an increasingly granular category.

For instance, all UCC codes beginning with ‘200’ represent beverages. UCC codes beginning with ‘20011’ represents beer and ‘200111’ represents ‘beer and ale’ and ‘200112’ represents ‘nonalcoholic beer’ (https://www.bls.gov/cex/pumd/ce_pumd_interview_diary_dictionary.xlsx).

The diaries also contain a flag that indicates whether the product was purchased as a gift. The challenge is to predict that flag using other information in the diary entries.

This can be done based on the following considerations:

1. Some items are _less likely to be purchased as gifts_ than others (for instance, it is unlikely that toilet paper is ever purchased as a gift).

2. Items that diverge from the _usual consumption patterns_ are more likely to be gifts.

In total, there are three tables which we find interesting:

1. EXPD, which contains information on the _consumer expenditures_, including the target variable GIFT.

2. FMLD, which contains socio-demographic information on the _households_.

3. MEMD, which contains socio-demographic information on each _member of the households_.


### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

In [1]:
import datetime
import os
from urllib import request
import time
import zipfile

import matplotlib.pyplot as plt
%matplotlib inline

import getml

In [2]:
getml.engine.set_project("consumer_expenditures")



Connected to project 'consumer_expenditures'


## 1. Loading data

### 1.1 Download from source


In [3]:
conn = getml.database.connect_mysql(
    host="relational.fit.cvut.cz",
    dbname="ConsumerExpenditures",
    port=3306,
    user="guest",
    password="relational"
)

conn

Connection(conn_id='default',
           dbname='ConsumerExpenditures',
           dialect='mysql',
           host='relational.fit.cvut.cz',
           port=3306)

In [4]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if getml.data.exists(name):
        return getml.data.load_data_frame(name)
    data_frame = getml.data.DataFrame.from_db(
        name=name,
        table_name=name,
        conn=conn
    )
    data_frame.save()
    return data_frame

In [5]:
households = load_if_needed("HOUSEHOLDS")
household_members = load_if_needed("HOUSEHOLD_MEMBERS")
expenditures = load_if_needed("EXPENDITURES")

In [6]:
households

name,YEAR,INCOME_RANK,INCOME_RANK_1,INCOME_RANK_2,INCOME_RANK_3,INCOME_RANK_4,INCOME_RANK_5,INCOME_RANK_MEAN,AGE_REF,HOUSEHOLD_ID
role,unused_float,unused_float,unused_float,unused_float,unused_float,unused_float,unused_float,unused_float,unused_float,unused_string
0,2015,0.3044,0.1448,0.1427,0.1432,0.1422,0.1382,0.127,66,03111041
1,2015,0.3063,0.1462,0.1444,0.1446,0.1435,0.1395,0.1283,66,03111042
2,2015,0.6931,0.6222,0.6204,0.623,0.6131,0.6123,0.6207,48,03111051
3,2015,0.6926,0.6216,0.6198,0.6224,0.6125,0.6117,0.6201,48,03111052
4,2015,0.2817,0.113,0.1128,0.1098,0.1116,0.1092,0.0951,37,03111061
,...,...,...,...,...,...,...,...,...,...
56807,2019,0.4828,0.4106,0.3603,0.3958,0.377,0.3984,0.3769,67,04362582
56808,2019,0.6644,0.5975,0.6026,0.5949,0.596,0.6002,0.6,52,04362661
56809,2019,0.6639,0.597,0.6021,0.5944,0.5955,0.5997,0.5995,52,04362662


In [7]:
household_members

name,YEAR,AGE,HOUSEHOLD_ID,MARITAL,SEX,WORK_STATUS
role,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string
0,2015,66,03111041,1,1,NULL
1,2015,66,03111042,1,1,NULL
2,2015,56,03111091,1,1,NULL
3,2015,56,03111092,1,1,NULL
4,2015,50,03111111,1,1,1
,...,...,...,...,...,...
137350,2019,22,04362422,5,2,NULL
137351,2019,11,04362431,5,2,NULL
137352,2019,11,04362432,5,2,NULL


In [8]:
expenditures

name,YEAR,MONTH,COST,GIFT,IS_TRAINING,EXPENDITURE_ID,HOUSEHOLD_ID,PRODUCT_CODE
role,unused_float,unused_float,unused_float,unused_float,unused_float,unused_string,unused_string,unused_string
0,2015,1,3.89,0,1,1,03111041,010210
1,2015,1,4.66,0,1,10,03111041,120310
2,2015,2,9.79,0,1,100,03111051,190211
3,2015,2,2.95,0,1,1000,03111402,040510
4,2015,1,2.12,0,1,10000,03114161,190321
,...,...,...,...,...,...,...,...
2020629,2017,6,1.99,0,1,999995,03708582,150110
2020630,2017,6,3.619,0,1,999996,03708582,150110
2020631,2017,6,5.2727,0,1,999997,03708582,150211


### 1.2 Prepare data for getML

We now have to assign roles to the columns.

In [9]:
households.set_role("HOUSEHOLD_ID", getml.data.roles.join_key)
households.set_role(households.unused_float_names, getml.data.roles.numerical)

households

name,HOUSEHOLD_ID,YEAR,INCOME_RANK,INCOME_RANK_1,INCOME_RANK_2,INCOME_RANK_3,INCOME_RANK_4,INCOME_RANK_5,INCOME_RANK_MEAN,AGE_REF
role,join_key,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical
0,03111041,2015,0.3044,0.1448,0.1427,0.1432,0.1422,0.1382,0.127,66
1,03111042,2015,0.3063,0.1462,0.1444,0.1446,0.1435,0.1395,0.1283,66
2,03111051,2015,0.6931,0.6222,0.6204,0.623,0.6131,0.6123,0.6207,48
3,03111052,2015,0.6926,0.6216,0.6198,0.6224,0.6125,0.6117,0.6201,48
4,03111061,2015,0.2817,0.113,0.1128,0.1098,0.1116,0.1092,0.0951,37
,...,...,...,...,...,...,...,...,...,...
56807,04362582,2019,0.4828,0.4106,0.3603,0.3958,0.377,0.3984,0.3769,67
56808,04362661,2019,0.6644,0.5975,0.6026,0.5949,0.596,0.6002,0.6,52
56809,04362662,2019,0.6639,0.597,0.6021,0.5944,0.5955,0.5997,0.5995,52


In [10]:
household_members.set_role("HOUSEHOLD_ID", getml.data.roles.join_key)
household_members.set_role(household_members.unused_float_names, getml.data.roles.numerical)
household_members.set_role(household_members.unused_string_names, getml.data.roles.categorical)

household_members

name,HOUSEHOLD_ID,MARITAL,SEX,WORK_STATUS,YEAR,AGE
role,join_key,categorical,categorical,categorical,numerical,numerical
0,03111041,1,1,NULL,2015,66
1,03111042,1,1,NULL,2015,66
2,03111091,1,1,NULL,2015,56
3,03111092,1,1,NULL,2015,56
4,03111111,1,1,1,2015,50
,...,...,...,...,...,...
137350,04362422,5,2,NULL,2019,22
137351,04362431,5,2,NULL,2019,11
137352,04362432,5,2,NULL,2019,11


In [11]:
year = expenditures["YEAR"]
month = expenditures["MONTH"]

ts_strings = year + "/" + month

expenditures["TIME_STAMP"] = ts_strings.as_ts(["%Y/%n"])

In [12]:
expenditures.set_role("HOUSEHOLD_ID", getml.data.roles.join_key)
expenditures.set_role("GIFT", getml.data.roles.target)
expenditures.set_role("COST", getml.data.roles.numerical)
expenditures.set_role(["PRODUCT_CODE", "MONTH", "YEAR"], getml.data.roles.categorical)
expenditures.set_role("TIME_STAMP", getml.data.roles.time_stamp)

expenditures.set_unit("PRODUCT_CODE", "code")

expenditures

name,TIME_STAMP,HOUSEHOLD_ID,GIFT,MONTH,YEAR,PRODUCT_CODE,COST,IS_TRAINING,EXPENDITURE_ID
role,time_stamp,join_key,target,categorical,categorical,categorical,numerical,unused_float,unused_string
unit,time stamp,,,,,code,,,
0,2015-01-01,03111041,0,1,2015,010210,3.89,1,1
1,2015-01-01,03111041,0,1,2015,120310,4.66,1,10
2,2015-02-01,03111051,0,2,2015,190211,9.79,1,100
3,2015-02-01,03111402,0,2,2015,040510,2.95,1,1000
4,2015-01-01,03114161,0,1,2015,190321,2.12,1,10000
,...,...,...,...,...,...,...,...,...
2020629,2017-06-01,03708582,0,6,2017,150110,1.99,1,999995
2020630,2017-06-01,03708582,0,6,2017,150110,3.619,1,999996


In [13]:
split = expenditures.rowid.as_str().update(expenditures.IS_TRAINING == 1, "train").update(expenditures.IS_TRAINING == 0, "test")
split

,
0,train
1,train
2,train
3,train
4,train
,...


## 2. Predictive modeling

Enough with the data preparation. Let's get to the fun part: Extracting the features.

### 2.1 Defining the data model

First, we define the data model.

What we want to do is the following: 

1. We want to compare every expenditure made to all *expenditures by the same household* (EXPD).

2. We want to check out whether *certain kinds of items have been purchased as a gift in the past* (EXPD).

2. We want to aggregate all available information on the *individual members of the household* (MEMD).



In [14]:
star_schema = getml.data.StarSchema(alias="POPULATION", population=expenditures, split=split)

star_schema.join(
    expenditures,
    on="HOUSEHOLD_ID",
    time_stamps="TIME_STAMP"
)

star_schema.join(
    households,
    on="HOUSEHOLD_ID",
    relationship=getml.data.relationship.many_to_one,
)

star_schema.join(
    household_members,
    on="HOUSEHOLD_ID",
)

star_schema

### 2.2 Setting the hyperparameters

We use `XGBoost` as our predictor and `FastProp` (short for fast propsitionalization) to generate our features. You are free to play with the hyperparameters.

In [15]:
ucc1 = getml.preprocessors.Substring(0, 1, "code")
ucc2 = getml.preprocessors.Substring(0, 2, "code")
ucc3 = getml.preprocessors.Substring(0, 3, "code")
ucc4 = getml.preprocessors.Substring(0, 4, "code")
ucc5 = getml.preprocessors.Substring(0, 5, "code")

mapping = getml.preprocessors.Mapping(multithreading=False)

fast_prop = getml.feature_learning.FastProp(
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    num_threads=1,
    sampling_factor=0.1,
    num_features=100,
)

relboost = getml.feature_learning.Relboost(
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    num_threads=1,
    num_features=20,
)

feature_selector = getml.predictors.XGBoostClassifier()

predictor = getml.predictors.XGBoostClassifier(
    booster="gbtree",
    n_estimators=100,
    max_depth=7,
    reg_lambda=0.0,
    n_jobs=1
)

In [16]:
pipe1 = getml.pipeline.Pipeline(
    tags=["FastProp"],
    data_model=star_schema.data_model,
    share_selected_features=0.2,
    preprocessors=[ucc1, ucc2, ucc3, ucc4, ucc5, mapping],
    feature_learners=fast_prop,
    feature_selectors=feature_selector,
    predictors=predictor
)

pipe1

Pipeline(data_model='POPULATION',
         feature_learners=['FastProp'],
         feature_selectors=['XGBoostClassifier'],
         include_categorical=False,
         loss_function=None,
         peripheral=['EXPENDITURES', 'HOUSEHOLDS', 'HOUSEHOLD_MEMBERS'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Substring', 'Substring', 'Substring', 'Substring', 'Substring',
                        'Mapping'],
         share_selected_features=0.2,
         tags=['FastProp'])

In [17]:
pipe2 = getml.pipeline.Pipeline(
    tags=["Relboost"],
    data_model=star_schema.data_model,
    share_selected_features=0.9,
    preprocessors=[ucc1, ucc2, ucc3, ucc4, ucc5, mapping],
    feature_learners=relboost,
    feature_selectors=feature_selector,
    predictors=predictor
)

pipe2

Pipeline(data_model='POPULATION',
         feature_learners=['Relboost'],
         feature_selectors=['XGBoostClassifier'],
         include_categorical=False,
         loss_function=None,
         peripheral=['EXPENDITURES', 'HOUSEHOLDS', 'HOUSEHOLD_MEMBERS'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Substring', 'Substring', 'Substring', 'Substring', 'Substring',
                        'Mapping'],
         share_selected_features=0.9,
         tags=['Relboost'])

In [18]:
pipe3 = getml.pipeline.Pipeline(
    tags=["FastProp", "Relboost"],
    data_model=star_schema.data_model,
    share_selected_features=0.2,
    preprocessors=[ucc1, ucc2, ucc3, ucc4, ucc5, mapping],
    feature_learners=[fast_prop, relboost],
    feature_selectors=feature_selector,
    predictors=predictor
)

pipe3

Pipeline(data_model='POPULATION',
         feature_learners=['FastProp', 'Relboost'],
         feature_selectors=['XGBoostClassifier'],
         include_categorical=False,
         loss_function=None,
         peripheral=['EXPENDITURES', 'HOUSEHOLDS', 'HOUSEHOLD_MEMBERS'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Substring', 'Substring', 'Substring', 'Substring', 'Substring',
                        'Mapping'],
         share_selected_features=0.2,
         tags=['FastProp', 'Relboost'])

`.fit(...)` will automatically call `.check(...)`, but it is always a good idea to call `.check(...)` separately, so we still have time for some last-minute fixes.

### 2.3 Training the pipeline

OK, let's go:

In [19]:
pipe1.check(star_schema.train)

Checking data model...


Preprocessing...
[========================================] 100%

OK.


In [20]:
pipe1.fit(star_schema.train)

Checking data model...


OK.


Preprocessing...
[========================================] 100%

FastProp: Trying 778 features...
[========================================] 100%

FastProp: Building features...
[========================================] 100%

XGBoost: Training as feature selector...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:38m:1.526164



Pipeline(data_model='POPULATION',
         feature_learners=['FastProp'],
         feature_selectors=['XGBoostClassifier'],
         include_categorical=False,
         loss_function=None,
         peripheral=['EXPENDITURES', 'HOUSEHOLDS', 'HOUSEHOLD_MEMBERS'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Substring', 'Substring', 'Substring', 'Substring', 'Substring',
                        'Mapping'],
         share_selected_features=0.2,
         tags=['FastProp', 'container-WBFn4W'])

url: http://localhost:1709/#/getpipeline/consumer_expenditures/rGyMaq/0/

In [21]:
pipe2.check(star_schema.train)

Checking data model...


Preprocessing...
[========================================] 100%

OK.


In [22]:
pipe2.fit(star_schema.train)

Checking data model...


OK.


Preprocessing...
[========================================] 100%

Relboost: Training features...
[========================================] 100%

Relboost: Building features...
[========================================] 100%

XGBoost: Training as feature selector...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:41m:21.771572



Pipeline(data_model='POPULATION',
         feature_learners=['Relboost'],
         feature_selectors=['XGBoostClassifier'],
         include_categorical=False,
         loss_function=None,
         peripheral=['EXPENDITURES', 'HOUSEHOLDS', 'HOUSEHOLD_MEMBERS'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Substring', 'Substring', 'Substring', 'Substring', 'Substring',
                        'Mapping'],
         share_selected_features=0.9,
         tags=['Relboost', 'container-WBFn4W'])

url: http://localhost:1709/#/getpipeline/consumer_expenditures/OSZaJl/0/

In [23]:
pipe3.check(star_schema.train)

Checking data model...


Preprocessing...
[========================================] 100%

OK.


In [ ]:
pipe3.fit(star_schema.train)

Checking data model...


OK.


Preprocessing...
[========================================] 100%

Retrieving features (because a similar feature learner has already been fitted)...
[========================================] 100%

Retrieving features (because a similar feature learner has already been fitted)...
[========================================] 100%

FastProp: Building features...
[========================================] 100%

Relboost: Building features...


### 2.4 Evaluating the pipeline

We want to know how well we did. We will to an in-sample and an out-of-sample evaluation:

In [ ]:
pipe1.score(star_schema.test)

In [ ]:
pipe2.score(star_schema.test)

In [ ]:
pipe3.score(star_schema.test)

### 2.5 Studying the features

It is very important that we get an idea about the features that the propositionalization algorithm has produced.

In [ ]:
LENGTH=50

names, correlations = pipe1.features.correlations()

plt.subplots(figsize=(20, 10))

plt.bar(names[:LENGTH], correlations[:LENGTH], color='#6829c2')

plt.title("feature correlations")
plt.grid(True)
plt.xlabel("features")
plt.ylabel("correlations")
plt.xticks(rotation='vertical')

plt.show()

In [ ]:
LENGTH=50

names, correlations = pipe2.features.correlations()

plt.subplots(figsize=(20, 10))

plt.bar(names[:LENGTH], correlations[:LENGTH], color='#6829c2')

plt.title("feature correlations")
plt.grid(True)
plt.xlabel("features")
plt.ylabel("correlations")
plt.xticks(rotation='vertical')

plt.show()

We can express the features in SQLite3:

Because getML uses a feature learning approach, the concept of feature importances can also be carried over to the individual columns.

In [ ]:
names, importances = pipe1.columns.importances()

plt.subplots(figsize=(20, 10))

plt.bar(names, importances, color='#6829c2')

plt.title("column importances")
plt.grid(True)
plt.xlabel("columns")
plt.ylabel("importance")
plt.xticks(rotation='vertical')

plt.show()

In [ ]:
names, importances = pipe2.columns.importances()

plt.subplots(figsize=(20, 10))

plt.bar(names, importances, color='#6829c2')

plt.title("column importances")
plt.grid(True)
plt.xlabel("columns")
plt.ylabel("importance")
plt.xticks(rotation='vertical')

plt.show()

## 3. Conclusion

In this notebook, we have shown how you can use relational learning to predict whether items were purchased as a gift. We did this to highlight the importance of relational learning. Relational learning can be used in many real-world data science applications, but unfortunately most data scientists don't even know what relation learning is.

If you want to learn more about getML, check out the [documentation](https://getml.com/product).

# Next Steps
This tutorial went through the basics of applying getML to a moderately-sized retail data set. 

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.